In [1]:
import os
import pandas as pd
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.metrics import accuracy_score
from collections import Counter
import pickle

In [2]:
# Path to the CSV file (adjust to your path)
csv_file = "C:/Users/HP/Desktop/image_dataset.csv"

In [3]:
# Function to load and preprocess images
def load_image(image_path, size=(64, 64)):
    try:
        img = Image.open(image_path).convert('RGB')
        img = img.resize(size)
        img_array = np.array(img).flatten()
        return img_array
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None

In [4]:
# Preprocess all images and extract features
X = []
y = []

In [5]:
# Load the CSV file
df = pd.read_csv(csv_file)
for index, row in df.iterrows():
    image_path = row['Filepath']
    image_array = load_image(image_path)
    if image_array is not None:
        X.append(image_array)
        y.append(row['Subfolder'])

X = np.array(X)
y = np.array(y)

In [6]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [7]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [8]:
# Train the model
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [9]:
# Save the trained XGBoost model using pickle
with open('Reset_classify_model.pkl', 'wb') as f:
    pickle.dump(xgb_model, f)

# Save the LabelEncoder using pickle
with open('reset_label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

In [10]:
# Function to reset the model
def reset_model(model_path):
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    return model

In [11]:
# Function to classify with consistency and rejection
def classify_with_consistency_and_rejection(image_path, model_path, label_encoder, num_iterations=10, threshold=0.7):
    try:
        # Reset model before prediction
        model = reset_model(model_path)
        
        # Load and preprocess the new image
        img = Image.open(image_path).convert('RGB')
        img = img.resize((64, 64))
        img_array = np.array(img).flatten()
        img_array = np.expand_dims(img_array, axis=0)
        
        # Collect multiple predictions
        predictions = []
        for _ in range(num_iterations):
            probabilities = model.predict_proba(img_array)[0]
            max_prob = np.max(probabilities)
            predicted_class_index = np.argmax(probabilities)
            
            if max_prob < threshold:
                predictions.append("Unknown/Unrelated pose")
            else:
                predicted_label = label_encoder.inverse_transform([predicted_class_index])[0]
                predictions.append(predicted_label)
        
        most_common_prediction = Counter(predictions).most_common(1)[0][0]
        return most_common_prediction
    except Exception as e:
        return f"Error during classification: {e}"

In [12]:
# Example usage
image_path = 'C:/Users/HP/Desktop/Images/InCorrect/00000041.jpg'
predicted_pose = classify_with_consistency_and_rejection(
    image_path, 'Reset_classify_model.pkl', label_encoder, num_iterations=10, threshold=0.7
)

print(f"The predicted pose for the given image is: {predicted_pose}")

The predicted pose for the given image is: Warrior2


In [13]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Input
import joblib

# Load your trained XGBoost model
xgb_model = joblib.load('Reset_classify_model.pkl')

# Create a Keras model to save in .h5 format
def create_keras_model(input_shape, num_classes):
    model = Sequential()
    model.add(Input(shape=(input_shape,)))  # Use Input layer for defining input shape
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))  # Softmax for multi-class
    return model

# Assuming X has been loaded and y_encoded is available
num_classes = len(np.unique(y_encoded))  # Number of classes in your dataset
input_shape = X.shape[1]  # Number of features

# Create Keras model
keras_model = create_keras_model(input_shape, num_classes)

# Compile the model (you may adjust loss and optimizer as needed)
keras_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Note: You may need to manually set the weights of the Keras model to those of the XGBoost model.
# This is non-trivial and may require additional steps, as Keras and XGBoost are different architectures.

# Save the Keras model in .h5 format
keras_model.save('C:/Users/HP/Reset_yoga_classify_model.h5')